In [2]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv


In [3]:
load_dotenv()

True

In [5]:
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.5)

In [8]:
prompt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}'
)

prompt = prompt_template.format(adjective='happy', content='brazil')

response = model.invoke(prompt)

print(response)



Why did the Brazilian soccer team bring a parrot to the game?

Because it was their wingman! 


## ChatPromptTemplate: The prompt to chat models is a list of chat messages

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [13]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpfull AI bot. Your name is {name}.'),
        ('human', 'Hello how are you doing'),
        ('ai', 'I\'m doing well, thanks!'),
        ('human', '{user_input}')
    ]
)

prompt = chat_template.format_messages(name='Bob', user_input='What is your name and what are you good at?')

response = model.invoke(prompt)

print(response)


AI: My name is Bob and I am an AI designed to assist and provide helpful responses to users. I am also good at processing and analyzing large amounts of data quickly and accurately. Is there something specific you would like me to help you with?


## Various ways of formatting System/Human/Ai prompts

In [14]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assiatnt that re-writes the user's text to"
                "sound more upbeat"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}")
    ]
)

prompt = chat_template.format_messages(text="I don't like eating tasty things")

response = model.invoke(prompt)

print(response)

.

System: I love indulging in delicious treats!


## Providing a context along the prompt

In [15]:
prompt = """Answer teh question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".vscode/

Context: Lage Language models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models 
can be acessed via Hugging Face's transformers library, via OpenAI
using the 'openai' library, and via Cohere using the `cohere` library.

Question: Which libraries and model providers offer LLMs?

Answer:"""

print(model.invoke(prompt))

 Hugging Face's transformers library, OpenAI's 'openai' library, and Cohere's `cohere` library.


## Langchain FewShotPromptTemplate

In [16]:
from langchain import FewShotPromptTemplate

In [19]:
# create examples
examples = [
    {
        'query': 'How are you?',
        'answer': 'I can\'t complain bu sometimes I still do.'
    },
    {
        'query': 'What times is it?',
        'answer': 'It\'s to get a watch.'
    }
]

example_template = """
User: {query}
AI: {answer}
"""

In [23]:
# create a prompt exmaple from above template
example_prompt = PromptTemplate(
    input_variables=['query', 'answer'],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the preffix is our instructions
prefix = """The following are experts from conversations with AI assitant.
The assitant is typically sarcastic anwitty, producing
creative and funny responses to the users questions. Here are some 
examples:"""

# and the suffix our user input and output indicator
suffix = """
User: {query}
AI:"""

In [26]:
few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator='\n\n'
)

query = 'Qhat movie should Iwatch evening?'

print(few_shot_template.format(query=query))

The following are experts from conversations with AI assitant.
The assitant is typically sarcastic anwitty, producing
creative and funny responses to the users questions. Here are some 
examples:


User: How are you?
AI: I can't complain bu sometimes I still do.



User: What times is it?
AI: It's to get a watch.



User: Qhat movie should Iwatch evening?
AI:


In [27]:
chain = few_shot_template | model

chain.invoke({'query': query})

' How about "The Shawshank Redemption"? It\'s a classic and I\'m sure you haven\'t seen it for the 100th time yet.'